# Dataset
Modifica e caricamento del dataset.\
Il dataset usato in questo caso è il dataset [COVIDx CXR-4](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2).\
Esso deve esser messo dentro la cartella 'datasets' al di fuori di questa cartella.

Questo notebook serve a modificare il dataset e a creare una cache contenente le immagini già pre-elaborate.\
Queste sono le azioni fatte ad ogni immagine:
- Caricamento dalla cartella del dataset
- Rimozione dei canali di colore \(alcune immagini hanno per esempio delle scritte rosse\); quindi ogni immagine è in scala di grigio.
- Salvataggio in float

Ogni immagine poi è accoppiata ad un valore 0 o 1 in base alla propria classe di appartenenza.

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from keras.api.preprocessing.image import load_img, img_to_array

# Define the dataset and cache paths
directory = '../../datasets/covid_cxr'
cache = f"{directory}_cache.npy"
types = ['train', 'val', 'test']
size = (224, 224)

In [2]:
# Load all the names of the images in one pandas
all_files = []
for t in types:
    df = pd.read_csv(f"{directory}/{t}.txt", delimiter=' ', header=None)
    df[1] = df[1].apply(lambda x: f"{directory}/{t}/{x}")
    all_files.append(df)

df = pd.concat(all_files)
df.columns = ['id', 'filename', 'class', 'source']

In [3]:
# Transform all the images into a numpy array
images = df['filename']
np_images = np.zeros((len(images), *size, 1), dtype=np.uint8)
for i, img_name in enumerate(tqdm(images)):
    img = load_img(img_name, target_size=size, color_mode='grayscale')
    img = img_to_array(img, dtype="uint8")
    np_images[i] = img.reshape((*size, 1))

100%|██████████| 84818/84818 [06:45<00:00, 209.04it/s]


In [4]:
# Convert the classes to a numpy array
predictions = np.array(pd.factorize(df['class'])[0])

In [ ]:
# Split the dataset into train, validation, and test sets
train_tot = len(all_files[0])
val_tot = train_tot + len(all_files[1])
test_tot = val_tot + len(all_files[2])

# Create a dictionary to store the dataset
dataset = {}
dataset['train'] = (np_images[:train_tot], predictions[:train_tot])
dataset['val'] = (np_images[train_tot:val_tot], predictions[train_tot:val_tot])
dataset['test'] = (np_images[val_tot:test_tot], predictions[val_tot:test_tot])

# Save the dataset to a cache file
np.save(cache, dataset)